In [5]:
# passo a passo


# passo 0: entender a empresa e o seu desafio
# passo 1: impotar a base de dados
import pandas as pd
import numpy as np
import json


tabela = pd.read_csv('treino.csv')
display(tabela)


FileNotFoundError: [Errno 2] No such file or directory: 'treino.csv'

In [ ]:
# passo 2: preparar a base de dados para a IA
tabela.info()

# y e quem eu quero prever
y = tabela['target']

# x e o que eu tenho para prever
x = tabela.drop(columns=['id','target'])

#separar dados de treino e dados de teste
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y)

# comferindo a calidade dos dados
from sklearn.preprocessing import StandardScaler

# Criar o normalizador (usando StandardScaler, pois funciona bem para SVM e redes neurais)
scaler = StandardScaler()

# Normalizar apenas para os modelos que precisam
x_treino_normalizado = scaler.fit_transform(x_treino)
x_teste_normalizado = scaler.transform(x_teste)



In [ ]:
# passo 3: treinar a IA
# criar modelo

#distancia
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#importar metricas
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# --- Ajuste refinado para KNN ---
param_grid_knn = {
    'n_neighbors': [4, 5, 6, 7, 8],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid_knn,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1
)
grid_knn.fit(x_treino_normalizado, y_treino)
best_knn_params = grid_knn.best_params_
print("Melhores parâmetros KNN:", best_knn_params)

# Treinamento final com os melhores parâmetros para KNN
modelo_knn = KNeighborsClassifier(**best_knn_params)
modelo_knn.fit(x_treino_normalizado, y_treino)




In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Grade de parâmetros reduzida
param_grid_svm = {
    'C': np.logspace(-1, 2, 4), # Menos valores para C
    'kernel': ['rbf'], # Kernel linear se apropriado
    'gamma': ['scale', 0.1, 1], # Menos valores para gamma
}

# RandomizedSearchCV para explorar o espaço de parâmetros de forma mais eficiente
grid_svm = RandomizedSearchCV(
    estimator=SVC(class_weight='balanced', probability=True, random_state=42),
    param_distributions=param_grid_svm, # Use param_distributions em vez de param_grid
    n_iter=10, # Número de iterações
    cv=3, # Número reduzido de folds de validação cruzada
    scoring='f1_macro',
    n_jobs=-1,
    random_state=42
)

grid_svm.fit(x_treino_normalizado, y_treino)
best_svm_params = grid_svm.best_params_
print("Melhores parâmetros SVM:", best_svm_params)

# Treinamento final com os melhores parâmetros para SVM
modelo_svm = SVC(**best_svm_params, class_weight='balanced', probability=True, random_state=42)
modelo_svm.fit(x_treino_normalizado, y_treino)

In [ ]:
# Avaliação dos modelos
previsao_knn = modelo_knn.predict(x_teste_normalizado)
previsao_svm = modelo_svm.predict(x_teste_normalizado)

print(classification_report(y_teste, previsao_knn))
print(classification_report(y_teste, previsao_svm))

In [ ]:
# Criar um dicionário com os melhores hiperparâmetros
melhores_hiperparametros = {
    'KNN': best_knn_params,
    'SVM': best_svm_params
}

import json

# Salvar os hiperparâmetros em um arquivo JSON
with open('melhores_hiperparametros1.json', 'w', encoding='utf-8') as arquivo_json:
    json.dump(melhores_hiperparametros, arquivo_json, indent=4, ensure_ascii=False)

print("Hiperparâmetros salvos em formato JSON com sucesso!")

In [ ]:
# qual o melhor modelo de IA com ajustes

previsao_knn = modelo_knn.predict(x_teste_normalizado)
previsao_svm = modelo_svm.predict(x_teste_normalizado)

print(classification_report(y_teste, previsao_knn))
print(classification_report(y_teste, previsao_svm))

print(confusion_matrix(y_teste, previsao_knn))
print(confusion_matrix(y_teste, previsao_svm))

In [6]:
# passo 5: usar o melhor modelo para faser a previsao
# qual o melhor modelo?

#importar tabela de novos cliemtes
tabela_para_teste = pd.read_csv('teste.csv')

# Armazena a coluna 'id' separadamente
ids = tabela_para_teste["id"]

# Remove a coluna 'id' antes da previsão
tabela_para_teste_sem_id = tabela_para_teste.drop(columns=['id'], errors='ignore')

# Faz a previsão
nova_previsao_knn = modelo_knn.predict(tabela_para_teste_sem_id)
nova_previsao_svm = modelo_svm.predict(tabela_para_teste_sem_id)

# Criar um DataFrame com os IDs e as previsões
resultado_knn = pd.DataFrame({"id": ids, "target": nova_previsao_knn})
resultado_svm = pd.DataFrame({"id": ids, "target": nova_previsao_svm})
display(tabela_para_teste)

display(nova_previsao_knn)
display(nova_previsao_svm)

# Exibir o resultado
display(resultado_knn)
display(resultado_svm)

# Salvando para CSV
df_knn = pd.DataFrame(resultado_knn)

df_knn.to_csv("resposta.knn.csv", index=False, header=True, encoding="utf-8")

df_svm = pd.DataFrame(resultado_svm)

df_svm.to_csv("resposta.svm.csv", index=False, header=True, encoding="utf-8")


FileNotFoundError: [Errno 2] No such file or directory: 'teste.csv'

In [ ]:
import pickle

# Salvando o modelo treinado
with open('modelo_knn_treinado.pkl', 'wb') as f:
    pickle.dump(modelo_knn, f)

print("Modelo salvo com sucesso!")
with open('modelo_svm_treinado.pkl', 'wb') as f:
    pickle.dump(modelo_svm, f)

print("Modelo salvo com sucesso!")
# Carregando o modelo salvo
with open('modelo_knn_treinado.pkl', 'rb') as f:
    modelo_knn_carregado = pickle.load(f)
with open('modelo_svm_treinado.pkl', 'rb') as f:
    modelo_svm_carregado = pickle.load(f)
# Agora você pode usar o modelo carregado para fazer previsões
y_pred_knn = modelo_knn_carregado.predict(x_teste_normalizado)
y_pred_svm = modelo_svm_carregado.predict(x_teste_normalizado)
